In [1]:
from datasets import load_dataset
import datasets
import pandas as pd
print(dir(datasets))

['Array2D', 'Array3D', 'Array4D', 'Array5D', 'ArrowBasedBuilder', 'Audio', 'BuilderConfig', 'ClassLabel', 'Column', 'Dataset', 'DatasetBuilder', 'DatasetDict', 'DatasetInfo', 'DownloadConfig', 'DownloadManager', 'DownloadMode', 'Features', 'GeneratorBasedBuilder', 'Image', 'IterableColumn', 'IterableDataset', 'IterableDatasetDict', 'LargeList', 'List', 'NamedSplit', 'NamedSplitAll', 'Pdf', 'ReadInstruction', 'Sequence', 'Split', 'SplitBase', 'SplitDict', 'SplitGenerator', 'SplitInfo', 'StreamingDownloadManager', 'SubSplitInfo', 'Translation', 'TranslationVariableLanguages', 'Value', 'VerificationMode', 'Version', 'Video', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'are_progress_bars_disabled', 'arrow_dataset', 'arrow_reader', 'arrow_writer', 'builder', 'combine', 'concatenate_datasets', 'config', 'data_files', 'dataset_dict', 'disable_caching', 'disable_progress_bar', 'disable_progress_bars', 'doc

In [2]:
import os
os.environ["WANDB_MODE"] = "offline"

In [3]:
data=load_dataset('SetFit/bbc-news')
print(data['train'][:5])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/880 [00:00<?, ?B/s]

train.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1225 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'text': ['wales want rugby league training wales could follow england s lead by training with a rugby league club.  england have already had a three-day session with leeds rhinos  and wales are thought to be interested in a similar clinic with rivals st helens. saints coach ian millward has given his approval  but if it does happen it is unlikely to be this season. saints have a week s training in portugal next week  while wales will play england in the opening six nations match on 5 february.  we have had an approach from wales   confirmed a saints spokesman.  it s in the very early stages but it is something we are giving serious consideration to.  st helens  who are proud of their welsh connections  are obvious partners for the welsh rugby union  despite a spat in 2001 over the collapse of kieron cunningham s proposed £500 000 move to union side swansea. a similar cross-code deal that took iestyn harris from leeds to cardiff in 2001 did go through  before the talented stand-off ret

In [4]:
train_df = pd.DataFrame(data["train"])
test_df = pd.DataFrame(data["test"])

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

print(train_df.head())
print(test_df.head())


Train shape: (1225, 3)
Test shape: (1000, 3)
                                                text  label     label_text
0  wales want rugby league training wales could f...      2          sport
1  china aviation seeks rescue deal scandal-hit j...      1       business
2  rock band u2 break ticket record u2 have smash...      3  entertainment
3  markets signal brazilian recovery the brazilia...      1       business
4  tough rules for ringtone sellers firms that fl...      0           tech
                                                text  label     label_text
0  carry on star patsy rowlands dies actress pats...      3  entertainment
1  sydney to host north v south game sydney will ...      2          sport
2  uk coal plunges into deeper loss shares in uk ...      1       business
3  blair joins school sailing trip the prime mini...      4       politics
4  bath faced with tindall ultimatum mike tindall...      2          sport


In [5]:
# [(i,v) for i,v in enumerate(train_df['label_text'])]

In [6]:
label2id = {label: i for i, label in enumerate(train_df["label_text"].unique())}
id2label = {v: k for k, v in label2id.items()}

train_df["label"] = train_df["label_text"].map(label2id)
test_df["label"]  = test_df["label_text"].map(label2id)


In [7]:
train_df.head()

,text,label,label_text
0,wales want rugby league training wales could f...,0,sport
1,china aviation seeks rescue deal scandal-hit j...,1,business
2,rock band u2 break ticket record u2 have smash...,2,entertainment
3,markets signal brazilian recovery the brazilia...,1,business
4,tough rules for ringtone sellers firms that fl...,3,tech


In [8]:
test_df.head()

,text,label,label_text
0,carry on star patsy rowlands dies actress pats...,2,entertainment
1,sydney to host north v south game sydney will ...,0,sport
2,uk coal plunges into deeper loss shares in uk ...,1,business
3,blair joins school sailing trip the prime mini...,4,politics
4,bath faced with tindall ultimatum mike tindall...,0,sport


In [9]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(
    train_df["text"].tolist(),
    truncation=True,
    padding=True,
    max_length=256
)

test_encodings = tokenizer(
    test_df["text"].tolist(),
    truncation=True,
    padding=True,
    max_length=256
)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
import torch

class BBCDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)

train_dataset = BBCDataset(train_encodings, train_df["label"].tolist())
test_dataset  = BBCDataset(test_encodings,  test_df["label"].tolist())


In [11]:
from transformers import BertForSequenceClassification, TrainingArguments, Trainer

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)


⚙️  Running in WANDB offline mode


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Freeze all BERT encoder layers
for param in model.bert.parameters():
    param.requires_grad = False

# Only the classification head (model.classifier) will be trained


In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bbc_bert_linear_probe",
    num_train_epochs=15,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    report_to=[],  # disables W&B and other loggers
)


In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # your PyTorch dataset
    eval_dataset=test_dataset      # your PyTorch dataset
)
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.607000,1.578608
2,1.579400,1.559582
3,1.562000,1.545086
4,1.544800,1.530202
5,1.535100,1.518444
6,1.522600,1.506794
7,1.509200,1.498126
8,1.506200,1.489643
9,1.498300,1.481974
10,1.497500,1.477132


TrainOutput(global_step=2310, training_loss=1.5188927390358664, metrics={'train_runtime': 575.8631, 'train_samples_per_second': 31.909, 'train_steps_per_second': 4.011, 'total_flos': 2417397933888000.0, 'train_loss': 1.5188927390358664, 'epoch': 15.0})

In [15]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [16]:
from transformers import BertForSequenceClassification, Trainer

# checkpoint_path = "./bbc_bert_linear_probe/checkpoint-1078"
# fine_tuned_model = BertForSequenceClassification.from_pretrained(checkpoint_path)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=test_dataset)
print(metrics)


{'eval_loss': 1.4642341136932373, 'eval_model_preparation_time': 0.0026, 'eval_accuracy': 0.419, 'eval_f1': 0.33420769432793657, 'eval_precision': 0.5769176821192054, 'eval_recall': 0.419, 'eval_runtime': 15.5371, 'eval_samples_per_second': 64.362, 'eval_steps_per_second': 8.045}


In [17]:
# Load the last fine-tuned checkpoint
model = BertForSequenceClassification.from_pretrained("./bbc_bert_linear_probe/checkpoint-2310")


# Training arguments
training_args = TrainingArguments(
    output_dir="./bbc_bert_linear_probe",
    num_train_epochs=15,             # you can increase epochs for more training
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,              # used internally by Trainer
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    report_to=[]
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Start training
trainer.train()

# Evaluate on test set
metrics = trainer.evaluate(eval_dataset=test_dataset)
print("Updated test metrics (pure linear probe):", metrics)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.529000,0.144759,0.966000,0.965994,0.967105,0.966000
2,0.055800,0.122496,0.962000,0.962316,0.964592,0.962000
3,0.018700,0.114109,0.976000,0.975967,0.976031,0.976000
4,0.007200,0.110072,0.979000,0.978986,0.979002,0.979000
5,0.006300,0.147449,0.972000,0.971963,0.972219,0.972000
6,0.002300,0.127416,0.976000,0.976008,0.976064,0.976000
7,0.001700,0.140235,0.975000,0.974983,0.975008,0.975000
8,0.001400,0.146404,0.973000,0.972963,0.973014,0.973000
9,0.001200,0.150520,0.973000,0.972963,0.973014,0.973000
10,0.001100,0.152692,0.973000,0.972963,0.973014,0.973000


Updated test metrics (pure linear probe): {'eval_loss': 0.1100718230009079, 'eval_accuracy': 0.979, 'eval_f1': 0.9789859595501077, 'eval_precision': 0.9790024305529778, 'eval_recall': 0.979, 'eval_runtime': 13.8459, 'eval_samples_per_second': 72.223, 'eval_steps_per_second': 4.55, 'epoch': 15.0}


In [18]:
train_df['label_text'].unique()

array(['sport', 'business', 'entertainment', 'tech', 'politics'],
      dtype=object)

In [19]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load your fine-tuned model (best checkpoint)
model = BertForSequenceClassification.from_pretrained(
    "./bbc_bert_linear_probe/checkpoint-2310"
)
model.eval()  # Set to evaluation mode

# Label mapping (BBC dataset usually has 5 classes)
label_names = list(train_df['label_text'].unique())

def predict(text):
    # Tokenize
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=256
    )

    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted class
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=1).item()

    # Return label name
    return label_names[predicted_class_id]

In [23]:
title=input("Enter your news title here: ")
predict(title)

Enter your news title here: china has the largest market of semiconductor


'politics'